## Práctica Guiada Requests y Beautiful Soup

En esta Práctica Guiada usaremos BeautifulSoup para descargar la información de la primera división de fútbol de Argentina de la página de ESPN:

    http://www.espn.com.ar/futbol/posiciones/_/liga/arg.

Debemos descargarla, extraerla, ordenarla y guardarla en un csv y/o un DataFrame para su uso. 
Primero importamos las librerias que necesitaremos.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

Para chequear que no hay problema scrapeando esta información, revisamos el archivo "robots.txt"

In [2]:
robots = 'http://www.espn.com.ar/' + 'robots.txt'
print(requests.get(robots).text)

User-agent: *
Disallow: *print?id
Disallow: /format/
Disallow: /nfl/format/player/design09/dropdown
Disallow: /nba/format/player/design09/dropdown
Disallow: /golf/deportes/format/player/design09/dropdown
Disallow: /extra/mma/deportes/format/player/design09/dropdown
Disallow: /extra/mma/format/player/design09/dropdown
Disallow: /rpm/deportes/format/player/design09/dropdown
Disallow: /golf/format/player/design09/dropdown
Disallow: /mlb/format/player/design09/dropdown
Disallow: /futbol/copamundial2010/
Disallow: /members/v3_1/login
Disallow: /insider/
Disallow: /videohub/video/clip?id
Disallow: /futbolint/noticias/
Disallow: /futbolargentino/columnas/
Disallow: /blogs/null
Disallow: /personalization/v3_1/personalization
Disallow: /futbol/equipo/null
Disallow: /mundial2006/
Disallow: /webslices/




No aparece nuestra URL, entonces procedemos.

Hacemos un requests "GET" a la URL con la tabla, y vemos el contenido de la respuesta en la Notebook

In [3]:
url = 'http://www.espn.com.ar/futbol/posiciones/_/liga/arg.1'
resp = requests.get(url)

In [4]:
#Vemos el contenido renderizado
display(HTML(resp.text))

In [6]:
soup = BeautifulSoup(resp.text, 'html.parser')

Podemos acceder a "tags" directamente

In [7]:
soup.title

<title>Posiciones de la Superliga Argentina</title>

Para trabajar con el contenido vamos a usar algunos de los siguientes métodos que tiene este objeto como:

- .findAll()  /  .find_all
- .find()
- .get()
- .get_text()

In [8]:
soup.head.link.get('href')

'http://www.espn.com.ar/futbol/posiciones/_/liga/arg.1'

In [9]:
#Podemos usar find() para encontrar la tabla

raw_table = soup.find('table', {'class':'standings has-team-logos'})

In [10]:
type(raw_table)

bs4.element.Tag

Esto nos devuelve un objeto "Tag" definido por la librería BeautifulSoup. Podemos ver sus atributos usando .attrs

In [11]:
raw_table.attrs

{'cellpadding': '0',
 'cellspacing': '0',
 'class': ['standings', 'has-team-logos'],
 'data-behavior': 'responsive_table',
 'data-fix-cols': '1',
 'data-mobile-force-responsive': 'true',
 'data-set-cell-heights': 'false',
 'data-text-contract': 'Contract table',
 'data-text-expand': 'Expand table'}

Dentro del atributo "children" encontramos todos los descendientes de ese nodo. En el caso de nuestra tabla, los descendientes son tags "tr", que corresponden a cada fila. Existe otro atributo llamado "descendants", que a diferencia del primero es recursivo. En este caso solo necesitamos los nódos directamente próximos, así que usamos el primer método.

Iteramos todas las filas y formamos una matriz con los datos, para luego cargarlos a un DataFrame.

In [12]:
rows = []
for row in raw_table.children:
    rows.append(row.get_text(separator= ','))
rows

['',
 '',
 '',
 '',
 '2017/18 Superliga,PJ,G,E,P,GF,GC,DIF,PTS',
 '1,Boca Juniors,CABJ,4,4,0,0,12,1,+11,12',
 '2,Unión de Santa Fe,USF,5,3,2,0,7,2,+5,11',
 '3,River Plate,CARP,4,3,1,0,8,3,+5,10',
 '4,Banfield,BAN,4,3,0,1,8,4,+4,9',
 '5,Patronato,CAP,4,3,0,1,7,5,+2,9',
 '6,Colón de Santa Fe,COL,4,2,2,0,5,2,+3,8',
 '7,San Lorenzo,SL,4,2,2,0,5,2,+3,8',
 '8,Huracán,HUR,5,2,2,1,5,4,+1,8',
 '9,Belgrano de Córdoba,BEL,5,2,2,1,4,3,+1,8',
 '10,Talleres de Córdoba,TDC,4,2,1,1,7,4,+3,7',
 '11,Vélez Sarsfield,VEL,4,2,1,1,5,4,+1,7',
 '12,Godoy Cruz de Mendoza,GCM,4,2,0,2,5,7,-2,6',
 '13,Lanús,LAN,4,2,0,2,4,6,-2,6',
 '14,Racing Club,RAC,4,1,2,1,5,3,+2,5',
 '15,Atlético Tucumán,TUC,5,1,2,2,4,6,-2,5',
 '16,Defensa y Justicia,DYJ,5,1,2,2,7,10,-3,5',
 '17,Independiente,IND,4,1,1,2,4,4,0,4',
 "18,Newell's Old Boys,NOB,4,1,1,2,3,3,0,4",
 '19,San Martín de San Juan,SMSJ,4,1,1,2,3,4,-1,4',
 '20,Gimnasia La Plata,GLP,4,1,1,2,6,8,-2,4',
 '21,Estudiantes La Plata,EST,4,1,1,2,3,5,-2,4',
 '22,Temperley,TEMP,4,1,

In [13]:
#Separamos las columnas, y desechamos las filas vacias
table = [row.split(',') for row in rows if len(row) > 1]

In [14]:
#Corregimos la tabla

table[0] =  ['Index', 'Name', 'Abbr'] + table[0][1:]
print(*table, sep = '\n')

['Index', 'Name', 'Abbr', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'DIF', 'PTS']
['1', 'Boca Juniors', 'CABJ', '4', '4', '0', '0', '12', '1', '+11', '12']
['2', 'Unión de Santa Fe', 'USF', '5', '3', '2', '0', '7', '2', '+5', '11']
['3', 'River Plate', 'CARP', '4', '3', '1', '0', '8', '3', '+5', '10']
['4', 'Banfield', 'BAN', '4', '3', '0', '1', '8', '4', '+4', '9']
['5', 'Patronato', 'CAP', '4', '3', '0', '1', '7', '5', '+2', '9']
['6', 'Colón de Santa Fe', 'COL', '4', '2', '2', '0', '5', '2', '+3', '8']
['7', 'San Lorenzo', 'SL', '4', '2', '2', '0', '5', '2', '+3', '8']
['8', 'Huracán', 'HUR', '5', '2', '2', '1', '5', '4', '+1', '8']
['9', 'Belgrano de Córdoba', 'BEL', '5', '2', '2', '1', '4', '3', '+1', '8']
['10', 'Talleres de Córdoba', 'TDC', '4', '2', '1', '1', '7', '4', '+3', '7']
['11', 'Vélez Sarsfield', 'VEL', '4', '2', '1', '1', '5', '4', '+1', '7']
['12', 'Godoy Cruz de Mendoza', 'GCM', '4', '2', '0', '2', '5', '7', '-2', '6']
['13', 'Lanús', 'LAN', '4', '2', '0', '2', '4', '6', '-2

Para guardar el archivo a un csv usando Python, usamos:

In [15]:
filename = 'tabla.csv'
with open(filename, 'w') as out:
    out.write('\n'.join([','.join(row) for row in table]))

In [16]:
df = pd.DataFrame(table[1:], columns= table[0])

In [17]:
df.columns

Index(['Index', 'Name', 'Abbr', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'DIF', 'PTS'], dtype='object')

In [18]:
df.set_index('Index', inplace=True)
df.head()

Name  Abbr PJ  G  E  P  GF GC  DIF PTS
Index                                                     
1           Boca Juniors  CABJ  4  4  0  0  12  1  +11  12
2      Unión de Santa Fe   USF  5  3  2  0   7  2   +5  11
3            River Plate  CARP  4  3  1  0   8  3   +5  10
4               Banfield   BAN  4  3  0  1   8  4   +4   9
5              Patronato   CAP  4  3  0  1   7  5   +2   9

Por último, podemos guardar el archivo usando pandas con:

In [19]:
filename = 'tabla.csv'
df.to_csv(filename)